In [102]:
import requests
import pandas as pd

In [103]:
def login(op, auth):
    
    with open('password'+str(op)+'.txt', 'r') as f:
        pw = f.read()
    with open('username'+str(op)+'.txt', 'r') as f:
        usrnm = f.read()
        
    headers = {'User-Agent': 'MyAPI/0.0.1'}
    
    res = requests.post('https://www.reddit.com/api/v1/access_token',
                        auth = auth,
                        data = {'grant_type': 'password',
                                'username': usrnm,
                                'password': pw}, 
                        headers = headers)
    TOKEN = res.json()['access_token']
    headers['Authorization'] = f'bearer {TOKEN}'
    
    return headers

In [106]:
with open('clientid.txt', 'r') as f:
    CLIENT_ID = f.read()
with open('secret.txt', 'r') as f:
    SECRET_KEY = f.read()   
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

headers_from = login(1, auth)
headers_to = login(2, auth)

## Fetch list of subreddits

In [107]:
df = pd.DataFrame()

res3 = requests.get('https://oauth.reddit.com/subreddits/mine/subscriber',
                   headers=headers_from, params = {'limit':'100'})
count = 100

while res3.json()['data']['dist'] != 0:
    
    for post in res3.json()['data']['children']:
        df = df.append({
            'URL': post['data']['url'],
            'Name': post['data']['name']        
        }, ignore_index=True)
        
    res3 = requests.get('https://oauth.reddit.com/subreddits/mine/subscriber',
                        headers=headers_from, params = {'after':df.iloc[-1]['Name'],
                                                        'limit':'100'})
    
    if res3.json()['data']['dist'] != 0:
        print('Fetched',count,'subreddits')
        count += 100
        
print('Finished fetching',len(df),'subreddits')

Fetched 100 subreddits
Fetched 200 subreddits
Fetched 300 subreddits
Fetched 400 subreddits
Fetched 500 subreddits
Fetched 600 subreddits
Fetched 700 subreddits
Fetched 800 subreddits
Fetched 900 subreddits
Fetched 1000 subreddits
Fetched 1100 subreddits
Fetched 1200 subreddits
Fetched 1300 subreddits
Finished fetching 1323 subreddits


# Rough Work

In [113]:
res = requests.get('https://oauth.reddit.com/api/subscribe',
                  headers = headers_to, params = {'skip_initial_defaults':'True',
                                               'action':'sub',
                                               'sr':'t5_2qjpg'})

In [114]:
res.json()

{'message': 'Not Found', 'error': 404}

In [111]:
res2 = requests.get('https://oauth.reddit.com/r/memes/about',
                   headers=headers)

In [112]:
res2.json()['data']['name']

't5_2qjpg'